# Clustering Solution Generator
### $Time$ $Series$ $3rd$ $Test$

$Vasco$ $Mergulhão$ $-$ $Jan$ $2023$

### Version 1:
This script loads a model and outputs a CSV ready to be analysed on the Dashboard

In [1]:
import os  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby
from datetime import timedelta, date
import plotly.graph_objects as go
import scipy
import math
from sklearn.cluster import KMeans

import random

import time
import datetime

import umap # UMAP library is responsible for ipywidgets warning!

import tensorflow as tf
from tensorflow import keras

import wandb
from wandb.keras import WandbCallback

C:\Users\ucesvpm\Anaconda3\envs\TensorFlow_PhD\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set Random Seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

---
# Script Variables

In [3]:
# MANUAL INPUTS
# MAKE SURE THEY ARE CORRECT
# Everything else is automated
dataset_name = 'Rwanda_10k_Set_1_w90'
Model_Name = 'FC_Small'
solution_name = 'fresh-sweep-2:v41'

# This loads model by name.
# Identify the best model by navigating the WandB Sweeps page.
# Find link to best model (and version)  in Artifacts -> Usage, and copy directory.
# .pb format is only accepted by Tensorflow, NOT Keras.
##############################################################################



In [4]:
# Uses name to navigate folders
dataset_folder = "_".join(dataset_name.split('_')[:-1]) #Takes out window length section
dataset_location = f'../Data/{dataset_folder}/{dataset_name}.csv'
dataset_window_key = dataset_name.split('_')[-1]

# Zcore Data Decision
zscore_data = True # Set to: [True/False]
zscore_data_done = False # Always set to False. Ensures its not normalized multiple times

# Project and Model Name
AE_Model_Type_dict = {'FC_Medium': 'FC_AE',
                      'FC_Small': 'FC_AE'
                     }
AE_Model_Type = AE_Model_Type_dict[f'{Model_Name}']

Project_Name = f'{AE_Model_Type}-{dataset_name}'


---
# Data Imports

In [5]:
Data = pd.read_csv(dataset_location)

In [6]:
Data.head()

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,9,0,2018-01-03,29.333333,28.333333,27.333333,614.732454,613.732454,24.333333,23.333333,...,9.333333,7.374942,7.333333,5.378137,5.333333,3.382164,2.386019,2.333333,1.333333,0.333333
1,9,1,2018-04-03,13.578796,12.587951,11.593588,10.603692,10.583356,9.583356,8.583356,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,6.682928,5.699398,5.708333,3.721609,2.729514
2,9,2,2018-07-02,2.708333,0.744861,0.708333,-7.000000,6.719641,5.719641,4.719641,...,3.599352,2.599352,1.599352,0.599352,-7.000000,7.657384,6.657384,5.657384,4.657384,3.657384
3,9,3,2018-09-30,2.657384,1.657384,0.657384,-7.000000,6.718287,5.718287,4.718287,...,7.715255,6.715255,5.715255,4.715255,3.715255,2.715255,1.715255,0.715255,-7.000000,-7.000000
4,9,4,2018-12-29,-7.000000,-7.000000,-7.000000,6.738970,5.738970,4.738970,3.738970,...,-7.000000,-7.000000,-7.000000,7.688646,6.688646,5.688646,4.688646,3.688646,2.688646,1.688646


---
# Z-Scoring Data

This is done on a row-by-row basis.<br>
Meaning, each window is normalized to its own Mean and Std.

In [7]:
def window_col_names(dataset_name, win_prefix = 'd'):
    # retriving window length
    window_len = int(dataset_name.split('_')[-1][1:]) # Gets _wXX part of name, then skips 'w' to get the number.
    # defining window column names
    window_cols = [None]*window_len
    for i  in range(window_len):
        window_cols[i] = f'{win_prefix}' + str(i+1)
        
    return window_cols
window_cols = window_col_names(dataset_name)     

In [8]:
def Zscore (df_in, all_neg_replace = -1):
    # retriving sets of columns
    window_cols = window_col_names(dataset_name)        
    cols = df_in.columns.to_list()
    other_cols = list(set(cols) - set(window_cols))
    
    # Zscore can't handle constant (flat) inputs
    # Therefore windows with all negative credit output NaNs
    # Instead these are excluded from zcoring and are manually scaled
    # Default scalling is  -7 => -1
    all_neg_replace = float(all_neg_replace)
    all_neg_index = df_in[(df_in[window_cols] == float(-7)).all(axis=1)].index
    positive_index = list(set(df_in.index.to_list()) - set(all_neg_index))
    
    if len(all_neg_index.to_list()) > 0:
        # Aux DF for replacing -7 with all_neg_replace (i.e, -1)
        df_allneg = df_in.iloc[all_neg_index].copy()
        df_allneg.loc[:, window_cols] = all_neg_replace

        # Zscoring only non-constant rows
        df_positive = df_in.iloc[positive_index].copy()    
        df_positive[window_cols] = scipy.stats.zscore(df_positive[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')

        df_zscore = pd.concat([df_positive, df_allneg])
        df_zscore.sort_values(by=['short_ID', 'window_ID'], inplace = True)

        # If indeces match, then the join was successful
        if df_zscore.index.to_list() == df_in.index.to_list():
            return df_zscore
        # Otherwise raise error
        else:
            print('Error Zscoring')
    else:
        df_zscore = pd.DataFrame(columns= cols)
        df_zscore[window_cols] = scipy.stats.zscore(df_in[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')
        df_zscore[other_cols] = df_in[other_cols]
        

    return df_zscore

In [9]:
if zscore_data == True and zscore_data_done == False:
    Data = Zscore(Data)
    zscore_data_done = True
    print('Data WAS Zscored')
    
elif zscore_data == True and zscore_data_done == True:
    print('Already WAS Zscored')
    
elif zscore_data == False:
    print('Data NOT Zscored')
    
else:
    print('Error')

Data WAS Zscored


---
# NaN Checks and Policy

In [10]:
def NaN_policy (df, method_fullrows = 'ffill', method_sparserows = 'ffill'):
    # This functions checks and corrects NaNs.
    # It has redundancy built into it to double check for NaNs.
    # And allows for different policies for sparse NaN values and rows full of NaNs.
    
    df = df.copy(deep=True)
    window_cols = window_col_names(dataset_name)
        
    if df[window_cols].isna().any().any():
        print('NaN values detected.')
        
        nan_exist = True
        i = 0   
        while nan_exist:
            
            # Makes sure the policy only iterates with one redundant round.
            if i >= 2:
                print('NaN Policy Failed')
                break            

            #Checking if there are rows with only NaNs
            if df[window_cols].isna().all(axis=1).any():
                n_nan_rows = len(df[df[window_cols].isna().all(axis=1)])
                print(f'There are {n_nan_rows} rows of all NaN values.')

                #Correcting only NaN rows 
                df.fillna(method= method_fullrows, axis = 0, inplace = True)
                if ~ df[window_cols].isna().all(axis=1).any():
                    print(f'Rows of NaN corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')
            
            #Checking if there are rows with any NaNs in them
            if df[window_cols].isna().any(axis=1).any():
                n_nan_rows = len(df[window_cols].isna().any(axis=1))
                print(f'There are {n_nan_rows} rows with some NaN values.')
                
                # Correcting NaN inside rows 
                df.fillna(method= method_sparserows, axis = 1, inplace = True)
                if ~ df[window_cols].isna().any(axis=1).any():
                    print(f'Rows with some NaN were corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')

            i += 1
            

    else:
        print('No NaNs detected')
        
    return df

In [11]:
Data = NaN_policy(Data)

No NaNs detected


---
---
# Loading Model

In [12]:
run = wandb.init()
model_artifact = run.use_artifact(f'vasco-phd/{Project_Name}/model-{solution_name}', type='model')
model_dir = model_artifact.download()
# local_model_dir = 'C:/Users/ucesvpm/OneDrive - University College London/PhD Project/Data Analytics/Time Series Clustering/Second Test/wandb/run-20221213_173041-jc918077'
loaded_autoencoder = tf.keras.models.load_model(model_dir)
run.finish()

wandb: Currently logged in as: vasco-mergulhao (vasco-phd). Use `wandb login --relogin` to force relogin


wandb:   4 of 4 files downloaded.  


In [13]:
loaded_autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dense (Dense)               (None, 200)               18200     
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 25)                5025      
                                                                 
 dense_3 (Dense)             (None, 200)               5200      
                                                                 
 dense_4 (Dense)             (None, 200)               40200     
                                                                 
 dense_5 (Dense)             (None, 90)                18090 

In [14]:
# loaded_autoencoder.get_config()

In [15]:
encoder_layers = int(len(loaded_autoencoder.layers)/2)

In [16]:
encoder_layers = int(len(loaded_autoencoder.layers)/2)
encoder = keras.models.Sequential(loaded_autoencoder.layers[:-encoder_layers])
# Finds latent space size
latent_layer_size = encoder.output_shape[-1]
encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               18200     
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 25)                5025      
                                                                 
Total params: 63,425
Trainable params: 63,425
Non-trainable params: 0
_________________________________________________________________


---
#  Reconstruction Error

In [17]:
# Calculates the Recontruction Profiles (Predictions)
y_pred = loaded_autoencoder.predict(Data[window_cols].to_numpy())

2575/2575 [==============================] - 4s 1ms/step


In [18]:
# Saves the Reconstructect windows
df_reconstruct = Data[['short_ID', 'window_ID']].copy(deep=True)
df_reconstruct[window_cols] = y_pred

In [19]:
# Calculating MSE per window
MSE = tf.keras.losses.MeanSquaredError(reduction='none')
MSE_values = MSE(Data[window_cols].to_numpy(), y_pred).numpy()
#Adding to reconstructed dataframe
df_reconstruct.insert(loc=2, column='MSE', value=MSE_values)

In [20]:
df_reconstruct.head()

,short_ID,window_ID,MSE,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,9,0,0.482980,-0.102187,-1.007373,1.050319,3.656369,3.045366,2.509418,2.137218,...,-0.072304,-0.032821,-0.036850,-0.138123,-0.185840,-0.399479,-0.304670,-0.343435,-0.313403,-0.328442
1,9,1,0.025875,1.351436,1.330900,1.086596,1.072830,0.901177,0.875316,0.687970,...,-1.309198,-1.341614,-1.355336,-1.271451,-1.883741,0.590185,0.501491,0.415888,0.206136,-0.103049
2,9,2,0.092484,-0.050814,-0.343420,-0.606102,-1.682097,1.295532,0.883522,0.606194,...,0.610883,0.299359,-0.132872,-0.603501,-1.849170,1.156427,0.930484,0.608403,0.469741,0.142674
3,9,3,0.114749,-0.005650,-0.378823,-0.981614,-1.707136,0.928653,0.869409,0.674384,...,1.484127,1.246733,1.025228,0.542506,0.297552,-0.117362,-0.631561,-1.221303,-1.899556,-2.151739
4,9,4,0.078083,-1.313143,-2.101218,-1.838945,1.056486,0.613684,0.336296,0.077689,...,-1.827554,-2.006069,-1.853642,1.158752,0.725102,0.781951,0.595702,0.190942,-0.017931,0.072777


---
---
# 2D UMAP

In [21]:
def UMAP_funct (data, dims = 2, mode = 'visualisation'):
    # this function allows you to switch from the settings optimazed from visualisation to clustering more easily
    start_time = time.time()
    if mode == 'visualisation':
        print(f'Processing UMAP {dims}D-Viz')
        umap_array = umap.UMAP(random_state=42, n_components = dims).fit_transform(data)
        print(f'Time: {np.round(time.time() - start_time,2)}[s]') 
        
        return umap_array
    
    elif mode == 'clustering':
        
        print(f'Processing UMAP {dims}D-Clust')
        # Settings from https://umap-learn.readthedocs.io/en/latest/clustering.html
        # General idea is larger n_neighbors to capture wider relationships, and smaller min_dist to keep points closer (better for density alg)
        umap_array = umap.UMAP(random_state=42,
                               n_components = dims,
                               n_neighbors=30,
                               min_dist=0.0
                              ).fit_transform(data)
        
        print(f'Time: {np.round(time.time() - start_time,2)}[s]') 
        
        return umap_array
    else:
        print(f'UMAP mode {mode} NOT recognized.')

In [22]:
# Calculating Latent Space Projection
encoded_data = encoder(Data[window_cols].to_numpy())
encoded_data = encoded_data.numpy()
df_encoded = pd.DataFrame(encoded_data)

In [23]:
#Calcs UMAP for Visual Purposes  
v_2D_umap = UMAP_funct (df_encoded, dims = 2, mode = 'visualisation')


Processing UMAP 2D-Viz
Time: 84.98[s]


In [24]:
# Creates DF for Clustering Solutions
df_sols  = Data.copy(deep=True)
df_sols.drop(window_cols, axis=1, inplace=True)

In [25]:
#Adds Dims to Dataframes
df_sols['UMAP_V1'] = v_2D_umap[:, 0]
df_sols['UMAP_V2'] = v_2D_umap[:, 1]

df_reconstruct.insert(loc=2, column='UMAP_V2', value=v_2D_umap[:, 1])
df_reconstruct.insert(loc=2, column='UMAP_V1', value=v_2D_umap[:, 0])

---
---
# Clustering Solutions

### kMeans

In [26]:
def kMeans_cluster(Encoded_Data, UMAP_mode = False):
    # Uses kMeans on encoded dataset.
    # UMAP_mode uses N2D-paper recomendation of UMAP encoded data to where k = Dims.
    # Of course, if latent size is smaller or equal to k, then no UMAP needed.
    
    # Output DF, only has solution columns
    df = pd.DataFrame()
    start_time = time.time()
    if UMAP_mode == True:
        print('Clustering on UMAP of Encoded Space')
        for k in range(2, 11):
            if k < latent_layer_size:
                # UMAPing to macthed k=dims with cluster friendly settings
                clust_umap = UMAP_funct (Encoded_Data, dims = k, mode = 'clustering')
                print(f'Processing kMeans for k={k}\n')
                partition = KMeans(n_clusters=k, random_state=42).fit(clust_umap)
                
                col_name = f'kMeans_k={k}'
                df[col_name] = partition.labels_ + 1

            else:
                print(f'k={k} is >= to Latent Space:{latent_layer_size}')
                print(f'Processing kMeans for k={k}')
                partition = KMeans(n_clusters=k, random_state=42).fit(Encoded_Data)
                
                col_name = f'kMeans_k={k}'
                df[col_name] = partition.labels_ + 1
                
    # Case with no UMAPing            
    else:
        print('Clustering on Encoded Space')
    
        # k-Means per se
        kmeans = [KMeans(n_clusters=k, random_state=42).fit(Encoded_Data)
                  for k in range(2, 11)]

        for partition in kmeans:
            cluster_k = partition.labels_.max() + 1
            col_name = f'kMeans_k={cluster_k}'
            df[col_name] = partition.labels_ + 1
        
    
    print(f'Total Time: {np.round(time.time() - start_time,2)}[s]')         
    return df

In [27]:
kMeans_sols = kMeans_cluster(df_encoded, UMAP_mode = False)

Clustering on Encoded Space
Total Time: 19.73[s]


In [28]:
df_sols = pd.concat([df_sols, kMeans_sols], axis=1)

### HDBSCAN
To Be Done

---
---
# Saving Outputs

### Clustering Solutions

In [29]:
df_sols.head()

,short_ID,window_ID,window_start_date,UMAP_V1,UMAP_V2,kMeans_k=2,kMeans_k=3,kMeans_k=4,kMeans_k=5,kMeans_k=6,kMeans_k=7,kMeans_k=8,kMeans_k=9,kMeans_k=10
0,9,0,2018-01-03,8.643910,4.357107,1,3,3,3,1,4,5,6,3
1,9,1,2018-04-03,7.141716,6.298301,1,3,3,3,6,4,5,1,3
2,9,2,2018-07-02,6.795166,2.582445,1,3,1,1,5,3,7,7,2
3,9,3,2018-09-30,6.397840,3.747205,1,3,1,1,5,3,5,6,6
4,9,4,2018-12-29,6.240857,3.084707,1,3,1,1,6,7,1,1,4


In [30]:
os.makedirs(f'../ModelResults/Clustering/{dataset_name}', exist_ok=True)  
solution_fileName = solution_name.replace(":", "_" )
solution_fileName = f'{Model_Name}_Latent-Space-{latent_layer_size}_{solution_fileName}.csv'
df_sols.to_csv(f'../ModelResults/Clustering/{dataset_name}/{solution_fileName}', index=False)  


### Auto-Encoder Recontruction

In [31]:
df_reconstruct.head()

,short_ID,window_ID,UMAP_V1,UMAP_V2,MSE,d1,d2,d3,d4,d5,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,9,0,8.643910,4.357107,0.482980,-0.102187,-1.007373,1.050319,3.656369,3.045366,...,-0.072304,-0.032821,-0.036850,-0.138123,-0.185840,-0.399479,-0.304670,-0.343435,-0.313403,-0.328442
1,9,1,7.141716,6.298301,0.025875,1.351436,1.330900,1.086596,1.072830,0.901177,...,-1.309198,-1.341614,-1.355336,-1.271451,-1.883741,0.590185,0.501491,0.415888,0.206136,-0.103049
2,9,2,6.795166,2.582445,0.092484,-0.050814,-0.343420,-0.606102,-1.682097,1.295532,...,0.610883,0.299359,-0.132872,-0.603501,-1.849170,1.156427,0.930484,0.608403,0.469741,0.142674
3,9,3,6.397840,3.747205,0.114749,-0.005650,-0.378823,-0.981614,-1.707136,0.928653,...,1.484127,1.246733,1.025228,0.542506,0.297552,-0.117362,-0.631561,-1.221303,-1.899556,-2.151739
4,9,4,6.240857,3.084707,0.078083,-1.313143,-2.101218,-1.838945,1.056486,0.613684,...,-1.827554,-2.006069,-1.853642,1.158752,0.725102,0.781951,0.595702,0.190942,-0.017931,0.072777


In [32]:
os.makedirs(f'../ModelResults/AE_Reconstruction/{dataset_name}', exist_ok=True)  
# reconstruction_fileName = solution_name.replace(":", "_" )
# reconstruction_fileName = f'{Model_Name}_Latent-Space-{latent_layer_size}_{reconstruction_fileName}.csv'
df_reconstruct.to_csv(f'../ModelResults/AE_Reconstruction/{dataset_name}/{solution_fileName}', index=False)  

wandb: While tearing down the service manager. The following error has occured: [WinError 10054] An existing connection was forcibly closed by the remote host
